In [1]:
import attr
import numpy as np
from thermo import Chemical
import warnings

from gibbs.models.ceos import PengRobinson, SoaveRedlichKwong
from gibbs.mixture import Mixture
from gibbs.stability_analysis import stability_test
from gibbs.equilibrium import calculate_equilibrium
from gibbs.utilities import convert_bar_to_Pa, convert_atm_to_Pa

warnings.filterwarnings('ignore')

In [2]:
@attr.s(auto_attribs=True)
class NichitaPR:
    mixture: Mixture
    bip: np.ndarray

    @property
    def model(self):
        return PengRobinson(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.mixture.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

## Two-phase vapor-liquid equilibria

### Eight-component mixture

In [3]:
methane = Chemical('methane')
ethane = Chemical('ethane')
propane = Chemical('propane')
nbutane = Chemical('n-butane')
npentane = Chemical('n-pentane')
nhexane = Chemical('n-hexane')

In [ ]:
z = np.array([0.6883, 0.0914, 0.0460, 0.0333, 0.0139, 0.0152, 0.0896, 0.0222])
omegas = np.array([methane.omega, ethane.omega, propane.omega, nbutane.omega, npentane.omega, nhexane.omega, 0.4019, 0.7987])
Tcs = np.array([methane.Tc, ethane.Tc, propane.Tc, nbutane.Tc, npentane.Tc, nhexane.Tc, 606.28, 825.67])
Pcs = np.array([methane.Pc, ethane.Pc, propane.Pc, nbutane.Pc, npentane.Pc, nhexane.Pc, convert_atm_to_Pa(25.42), convert_atm_to_Pa(14.39)])
mixture = Mixture(z, Tcs, Pcs, omegas)
kijs = np.zeros((8, 8))
kijs[0, 6] = kijs[6, 0] = 0.050
kijs[1, 6] = kijs[6, 1] = 0.040
kijs[2, 6] = kijs[6, 2] = 0.010
kijs[0, 7] = kijs[7, 0] = 0.090
kijs[1, 7] = kijs[7, 1] = 0.055
kijs[2, 7] = kijs[7, 2] = 0.010

model = NichitaPR(
    mixture=mixture,
    bip=kijs
)

In [ ]:
# NBVAL_SKIP
T = 353
P = convert_bar_to_Pa(385)

result = calculate_equilibrium(model, P, T, z, molar_base=1, number_of_trial_phases=2, workers=-1, compare_trial_phases=False)

result.X, result.F, result.reduced_gibbs_free_energy

## Three-phase vapor-liquid-liquid equilibria

### Case 4.3.1: Ternary mixture C$_1$/nC$_{16}$/CO$_2$

In [4]:
nhexadecane = Chemical('n-hexadecane')
carbon_dioxide = Chemical('carbon-dioxide')

In [5]:
z = np.array([0.05, 0.05, 0.90])
omegas = np.array([methane.omega, nhexadecane.omega, carbon_dioxide.omega])
Tcs = np.array([methane.Tc, nhexadecane.Tc, carbon_dioxide.Tc])
Pcs = np.array([methane.Pc, nhexadecane.Pc, carbon_dioxide.Pc])
mixture = Mixture(z, Tcs, Pcs, omegas)
kijs = np.array([
    [0.000, 0.078, 0.100],
    [0.078, 0.000, 0.125],
    [0.100, 0.125, 0.000]
])

model = NichitaPR(
    mixture=mixture,
    bip=kijs
)

In [6]:
# NBVAL_SKIP
T = 294.3
P = convert_bar_to_Pa(67)

result = calculate_equilibrium(model, P, T, z, molar_base=1, number_of_trial_phases=3)

result.X, result.F, result.reduced_gibbs_free_energy

(array([[3.77832365e-02, 3.20442358e-03, 9.59012340e-01],
        [7.64576597e-02, 3.96816762e-05, 9.23502659e-01],
        [2.79799743e-02, 3.64208774e-01, 6.07811252e-01]]),
 array([0.51778789, 0.34952195, 0.13269016]),
 14.24540622944931)